In [139]:
from datetime import datetime, timedelta

import polars as pl

df = pl.DataFrame(
    {
        "metric_nice_name": [
            "WaterTank3Volume-Current",
            "WaterTank3Capacity",
            "OilTank2Level",
            "OilTank2Volume-Current",
            "ChemicalTankLevel",
            "ChemicalTankCapacity",
        ],
        "timestamp": [
            datetime.now(),
            datetime.now(),
            datetime.now(),
            datetime.now(),
            datetime.now() - timedelta(days=9),
            datetime.now() - timedelta(days=9),
        ],
        "value": [10.0, 100, 20, 30, 40, 50],
    }
)
df

metric_nice_name,timestamp,value
str,datetime[μs],f64
"""WaterTank3Volu…",2024-05-08 22:37:11.659769,10.0
"""WaterTank3Capa…",2024-05-08 22:37:11.659771,100.0
"""OilTank2Level""",2024-05-08 22:37:11.659771,20.0
"""OilTank2Volume…",2024-05-08 22:37:11.659771,30.0
"""ChemicalTankLe…",2024-04-29 22:37:11.659771,40.0
"""ChemicalTankCa…",2024-04-29 22:37:11.659775,50.0


In [140]:
tank_type_col = "tank_type"
tank_number_col = "tank_number"
metric_measurement_col = "metric_measurement"

tank_types = ["Oil", "Water", "Chemical"]
metric_measurements = ["Level", "Volume-Current", "ID", "Capacity"]
metric_measurements_regex = "|".join(metric_measurements)
tank_types_regex = "|".join(tank_types)

regexi = f"(?<{tank_type_col}>{tank_types_regex})Tank(?<{tank_number_col}>[0-9]*)(?<{metric_measurement_col}>{metric_measurements_regex})"
df = df.with_columns(
    pl.col("metric_nice_name").str.extract_groups(regexi).alias("tank_capture_groups")
)
df

metric_nice_name,timestamp,value,tank_capture_groups
str,datetime[μs],f64,struct[3]
"""WaterTank3Volu…",2024-05-08 22:37:11.659769,10.0,"{""Water"",""3"",""Volume-Current""}"
"""WaterTank3Capa…",2024-05-08 22:37:11.659771,100.0,"{""Water"",""3"",""Capacity""}"
"""OilTank2Level""",2024-05-08 22:37:11.659771,20.0,"{""Oil"",""2"",""Level""}"
"""OilTank2Volume…",2024-05-08 22:37:11.659771,30.0,"{""Oil"",""2"",""Volume-Current""}"
"""ChemicalTankLe…",2024-04-29 22:37:11.659771,40.0,"{""Chemical"","""",""Level""}"
"""ChemicalTankCa…",2024-04-29 22:37:11.659775,50.0,"{""Chemical"","""",""Capacity""}"


In [141]:
df = df.unnest("tank_capture_groups")
df

metric_nice_name,timestamp,value,tank_type,tank_number,metric_measurement
str,datetime[μs],f64,str,str,str
"""WaterTank3Volu…",2024-05-08 22:37:11.659769,10.0,"""Water""","""3""","""Volume-Current…"
"""WaterTank3Capa…",2024-05-08 22:37:11.659771,100.0,"""Water""","""3""","""Capacity"""
"""OilTank2Level""",2024-05-08 22:37:11.659771,20.0,"""Oil""","""2""","""Level"""
"""OilTank2Volume…",2024-05-08 22:37:11.659771,30.0,"""Oil""","""2""","""Volume-Current…"
"""ChemicalTankLe…",2024-04-29 22:37:11.659771,40.0,"""Chemical""","""""","""Level"""
"""ChemicalTankCa…",2024-04-29 22:37:11.659775,50.0,"""Chemical""","""""","""Capacity"""


In [142]:
df = df.with_columns(pl.col(tank_number_col).cast(pl.UInt8, strict=False))
df

metric_nice_name,timestamp,value,tank_type,tank_number,metric_measurement
str,datetime[μs],f64,str,u8,str
"""WaterTank3Volu…",2024-05-08 22:37:11.659769,10.0,"""Water""",3,"""Volume-Current…"
"""WaterTank3Capa…",2024-05-08 22:37:11.659771,100.0,"""Water""",3,"""Capacity"""
"""OilTank2Level""",2024-05-08 22:37:11.659771,20.0,"""Oil""",2,"""Level"""
"""OilTank2Volume…",2024-05-08 22:37:11.659771,30.0,"""Oil""",2,"""Volume-Current…"
"""ChemicalTankLe…",2024-04-29 22:37:11.659771,40.0,"""Chemical""",null,"""Level"""
"""ChemicalTankCa…",2024-04-29 22:37:11.659775,50.0,"""Chemical""",null,"""Capacity"""


In [143]:
rt_metrics = ["Level", "Volume-Current"]

cutoff_datetime = datetime.now() - timedelta(days=7)
df = df.filter(
    pl.col(metric_measurement_col).is_in(rt_metrics).not_()
    | pl.col("timestamp").gt(cutoff_datetime)
)
df

metric_nice_name,timestamp,value,tank_type,tank_number,metric_measurement
str,datetime[μs],f64,str,u8,str
"""WaterTank3Volu…",2024-05-08 22:37:11.659769,10.0,"""Water""",3,"""Volume-Current…"
"""WaterTank3Capa…",2024-05-08 22:37:11.659771,100.0,"""Water""",3,"""Capacity"""
"""OilTank2Level""",2024-05-08 22:37:11.659771,20.0,"""Oil""",2,"""Level"""
"""OilTank2Volume…",2024-05-08 22:37:11.659771,30.0,"""Oil""",2,"""Volume-Current…"
"""ChemicalTankCa…",2024-04-29 22:37:11.659775,50.0,"""Chemical""",null,"""Capacity"""


In [144]:
tank_aggs = [
    pl.col("value")
    .filter(pl.col(metric_measurement_col) == metric)
    .first()
    .alias(metric)
    for metric in metric_measurements
]
tank_aggs.append(
    pl.col("timestamp").filter(pl.col(metric_measurement_col).is_in(rt_metrics)).first()
)
df = df.group_by([tank_type_col, tank_number_col]).agg(tank_aggs)
df

tank_type,tank_number,Level,Volume-Current,ID,Capacity,timestamp
str,u8,f64,f64,f64,f64,datetime[μs]
"""Water""",3,null,10.0,null,100.0,2024-05-08 22:37:11.659769
"""Oil""",2,20.0,30.0,null,null,2024-05-08 22:37:11.659771
"""Chemical""",null,null,null,null,50.0,null
